In [1]:
# Import modules
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [2]:
#load data and take out id columns
new_data = pd.read_csv('data/final_data.csv')

In [3]:
#numeric columns
num =['time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency',
'number_inpatient','number_diagnoses']
#normalize new_data numeric columns
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(new_data[num])
new_data[num] = transformer.transform(new_data[num])
new_data[num] = pd.DataFrame(new_data[num], columns= new_data[num].columns)

In [4]:
#Dummify
new_data = pd.get_dummies(new_data, drop_first = True)
new_data.shape

(84432, 151)

In [5]:
new_data.columns


Index(['admission_type_id', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses', 'target',
       ...
       'discharge_id_5', 'discharge_id_6', 'discharge_id_Other',
       'admission_id_17', 'admission_id_2', 'admission_id_4', 'admission_id_5',
       'admission_id_6', 'admission_id_7', 'admission_id_Other'],
      dtype='object', length=151)

In [6]:
# #enlabel the columns
# new_data = new_data
# new_data['admission_type_id'] = pd.Categorical(new_data.admission_type_id)
# new_data['discharge_disposition_id'] = pd.Categorical(new_data.discharge_disposition_id)
# new_data['admission_source_id'] = pd.Categorical(new_data.admission_source_id)

# cat_cols = list(new_data.select_dtypes(include=[object]).columns)
# for col in cat_cols:
#    new_data[col] = pd.Categorical(new_data[col])

# le = preprocessing.LabelEncoder()

# col_to_encode = new_data[list(new_data.select_dtypes(include=['category']).columns)]
# for col in col_to_encode:
#    new_data[col] = le.fit_transform(new_data[col])

In [7]:
#Up-sample Minority Class
from sklearn.utils import resample
majority = new_data[new_data.target==0]
minority = new_data[new_data.target==1]
minority_upsampled = resample(minority,replace=True,n_samples=74817,random_state=123)
new_data1 = pd.concat([majority, minority_upsampled])
new_data1.target.value_counts()

1    74817
0    74817
Name: target, dtype: int64

In [8]:
#Spliting to training
target = new_data1.target
X_train, X_test1, y_train, y_test1 = train_test_split(new_data1, target, test_size=0.25, stratify = target)
print(X_train.shape, y_train.shape)
print(X_test1.shape, y_test1.shape)

(112225, 151) (112225,)
(37409, 151) (37409,)


In [9]:
#Spliting to testing datasets
target = new_data.target
X_train1, X_test, y_train1, y_test = train_test_split(new_data, target, test_size=0.25, stratify = target)
print(X_train1.shape, y_train1.shape)
print(X_test.shape, y_test.shape)

(63324, 151) (63324,)
(21108, 151) (21108,)


In [10]:
#drop target from training
X_train = X_train.drop('target', axis=1)
X_test = X_test.drop('target', axis=1)

In [11]:
### Fit the model with training
model = GaussianNB()
model.fit(X_train, y_train)
## The score (accuracy for classification problems):
model.score(X_train, y_train)

0.5092002673201158

In [12]:
#AUC test  
from sklearn.metrics import roc_auc_score
upsample_AUC_train =roc_auc_score(y_test, model.predict(X_test))
upsample_AUC_train

0.508690738916747

In [13]:
#confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))


array([[  683, 18021],
       [   46,  2358]])